# Text-GAN Turkish word generation

In [1]:
using Knet, Test, Base.Iterators, Printf, LinearAlgebra, CuArrays, Random, IterTools, StatsBase

struct Charset
    c2i::Dict{Any,Int}
    i2c::Vector{Any}
    eow::Int
end

function Charset(charset::String; eow="")
    i2c = [ eow; [ c for c in charset ]  ]
    print(i2c)
    c2i = Dict( c => i for (i, c) in enumerate(i2c))
    return Charset(c2i, i2c, c2i[eow])
end

struct TextReader
    file::String
    charset::Charset
end

function Base.iterate(r::TextReader, s=nothing)
    s === nothing && (s = open(r.file))
    eof(s) && return close(s)
    return [ get(r.charset.c2i, c, r.charset.eow) for c in readline(s)], s
end

Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

struct WordsData
    src::TextReader        
    batchsize::Int         
    maxlength::Int         
    batchmajor::Bool       
    bucketwidth::Int    
    buckets::Vector        
end

function WordsData(src::TextReader; batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 2, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar length
    WordsData(src, batchsize, maxlength, batchmajor, bucketwidth, buckets)
end

Base.IteratorSize(::Type{WordsData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{WordsData}) = Base.HasEltype()
Base.eltype(::Type{WordsData}) = Array{Any,1}

function Base.iterate(d::WordsData, state=nothing)
    if state == 0 # When file is finished but buckets are partially full 
        for i in 1:length(d.buckets)
            if length(d.buckets[i]) > 0
                buc = d.buckets[i]
                d.buckets[i] = []
                return buc, state
            end
        end
        return nothing # Finish iteration
    end

    while true
        src_next = iterate(d.src, state)
        
        if src_next === nothing
            state = 0
            return iterate(d, state)
        end
        
        (src_word, src_state) = src_next
        state = src_state
        src_length = length(src_word)
        
        (src_length > d.maxlength) && continue

        i = Int(ceil(src_length / d.bucketwidth))
        i > length(d.buckets) && (i = length(d.buckets))

        push!(d.buckets[i], src_word)
        if length(d.buckets[i]) == d.batchsize
            buc = d.buckets[i]
            d.buckets[i] = []
            return buc, state
        end
    end
end

function arraybatch(d::WordsData, bucket)
    src_eow = d.src.charset.eow
    src_lengths = map(x -> length(x), bucket)
    max_length = max(src_lengths...)
    x = zeros(Int64, length(bucket), d.maxlength + 1) # default d.batchmajor is false

    for (i, v) in enumerate(bucket)
        to_be_added = fill(src_eow, d.maxlength - length(v))
        x[i,:] = [src_eow; v; to_be_added]
    end
    
    d.batchmajor && (x = x')
    return (x[:, 1:end-1], x[:, 2:end]) # to calculate nll on generators output directly
end

function readwordset(fname)
    words = []
    fi = open(fname)
    while !eof(fi)
        push!(words, readline(fi))
    end
    close(fi)
    words
end

function mask(a, pad)
    a = copy(a)
    for i in 1:size(a, 1)
        j = size(a,2)
        while a[i, j] == pad && j > 1
            if a[i, j - 1] == pad
                a[i, j] = 0
            end
            j -= 1
        end
    end
    return a
end

mask (generic function with 1 method)

### G/D Common Parts

In [2]:
struct Embed; w; end

function Embed(shape...)
    Embed(param(shape...))
end

get_z(shape...) = KnetArray(randn(Float32, shape...))

# this function is similar to gumble softmax, it is used to soften the one-hot-vector of the real samples
# tau -> normalization factor; the bigger the softer
function soften(A; dims=1, tau=2.0, norm_factor=0.01) 
    A = (A .+ norm_factor) ./ tau
    softmax(A; dims=dims)
end

# per-word loss (in this case per-batch loss)
function loss(model, data; average=true)
    l = 0
    n = 0
    a = 0
    for (x, y) in data
        v = model(x, y; average=false)
        l += v[1]
        n += v[2]
        a += (v[1] / v[2])
    end
    average && return a
    return l, n
end

loss (generic function with 1 method)

## CNN Discriminator

In [3]:
# This one to be used by DModel, takes weights of characters and reduce the embedding for each character
# this approach to avoid sampling or argmaxing over rnn's output
# (C, B, T) -> (T, E, 1, B)

# function (l::Embed)(x)
#     dims = size(x)
#     em = l.w * reshape(x, dims[1], dims[2] * dims[3]) # reshape for multiplication 
#     em = reshape(em, size(em, 1), dims[2], dims[3]) # reshape to original size
#     em = permutedims(em, [3, 1, 2])  # permute for CONV
#     em = reshape(em, dims[3], size(em, 2), 1, dims[2]) # Add one dim for CONV
# end

# struct Conv; w; b; f; p; end
# (c::Conv)(x) = (co=conv4(c.w, dropout(x,c.p)); c.f.(pool((co .+ c.b); window=(size(co, 1), size(co, 2)))))
# Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

# struct Dense; w; b; f; p; end
# (d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
# Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

# # Perform convolution then, global-max pooling and concatenate the output and feed it to sequential dense layer 
# mutable struct DisModel
#     charset::Charset
#     embed::Embed
#     filters
#     dense_layers
# end

# # This discriminator uses separate weights for its embedding layer
# function DisModel(charset, embeddingSize::Int, filters, denselayers)
#     Em = Embed(embeddingSize, length(charset.c2i))
#     DisModel(charset, Em, filters, denselayers)
# end

# function (c::DisModel)(x) # the input here is weights of the characters with shape (C, B, T)
#     em = c.embed(x)
#     filters_out = []
#     for f in c.filters
#         push!(filters_out, f(em))
#     end
#     max_out = cat(filters_out...;dims=3)
#     for l in c.dense_layers
#         max_out = l(max_out)
#     end
#     max_out
# end

# (c::DisModel)(x,y; average=true) = nll(c(x), y; average=average)


## Recurrent Discriminator

In [4]:
function (l::Embed)(x)
    dims = size(x)
    em = l.w * reshape(x, dims[1], dims[2] * dims[3]) # reshape for multiplication 
    em = reshape(em, size(em, 1), dims[2], dims[3]) # reshape to original size
end

struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 3-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

mutable struct DisModel
    charset::Charset
    embed::Embed
    rnn::RNN
    denselayers
end

# This discriminator uses separate weights for its embedding layer
function DisModel(charset, embeddingSize::Int, hidden, denselayers; layers=1, dropout=0)
    Em = Embed(embeddingSize, length(charset.c2i))
    rnn = RNN(embeddingSize, hidden; numLayers=layers, dropout=dropout)
    DisModel(charset, Em, rnn, denselayers)
end

function (c::DisModel)(x) # the input here is weights of the characters with shape (C, B, T)
    c.rnn.h, c.rnn.c = 0, 0
    em = c.embed(x)
    rnn_out = c.rnn(em)
    dims = size(rnn_out)
    rnn_out = reshape(rnn_out, :, dims[2] * dims[3] )
    for l in c.denselayers
        rnn_out = l(rnn_out)
    end
    reshape(rnn_out, :, dims[2], dims[3])
end

function (c::DisModel)(x, y, calculatereward::Int)
    scores = reshape(c(x), :, size(y, 1) * size(y, 2))
    labels = reshape(y, size(y, 1) * size(y, 2))
    losses = [ nll(scores[:, i], labels[i:i]) for i in 1:size(labels, 1) ]
end

function (c::DisModel)(x, y; average=true)
    scores = reshape(c(x), :, size(y, 1) * size(y, 2))
    labels = reshape(y, size(y, 1) * size(y, 2))
    return nll(scores, y; average=average)
end

## Generator

In [5]:
# concatenate z with embedding vectors, z -> (z_size, B), returns (E+z_size, B, T)
# this will be used to feed Z to generator at each timestep
function (l::Embed)(x, z)
    em = l.w[:, x]
    z_array = cat((z for i in 1:size(em, 3))...; dims=(3))
    cat(em, z_array; dims=(1))
end

# Generator model
struct GenModel
    embed::Embed
    rnn::RNN        
    dropout::Real
    charset::Charset
    projection::Embed
    disModel::DisModel
    maxlength::Int
    zsize::Int
end

function GenModel(esize::Int, zsize::Int, hidden::Int, charset::Charset, disModel::DisModel, maxlength::Int; layers=2, dropout=0)
    embed = Embed(esize, length(charset.i2c))
    rnn = RNN(zsize + esize, hidden; numLayers=layers, dropout=dropout)
    projection = Embed(hidden, length(charset.i2c))
    GenModel(embed, rnn, dropout, charset, projection, disModel, maxlength, zsize)
end

# This generator shares the projection layers weights of the discriminator for its projection layer
function GenModel(esize::Int, zsize::Int, charset::Charset, disModel::DisModel, maxlength::Int; layers=2, dropout=0)
    embed = Embed(esize, length(charset.i2c))
    rnn = RNN(zsize + esize, size(disModel.embed.w, 1); numLayers=layers, dropout=dropout)
    GenModel(embed, rnn, dropout, charset, disModel.embed, disModel, maxlength, zsize)
end

# function Z(s::GenModel, batchsize, timesteps)
#     z = get_z(s.zsize, batchsize, 1) # according to get_z(H, B, layers)
#     return cat([ z for i in 1:timesteps]...;dims=3)
# end


# # Generator forward pass using only Z as input, size(Z) -> inputsize, batchsize, sequencelength
# function (s::GenModel)(Z)
#     s.rnn.h, s.rnn.c = 0, 0
#     rnn_out = s.rnn(Z) 
#     dims = size(rnn_out)
#     output = s.projection.w' * dropout(reshape(rnn_out, dims[1], dims[2] * dims[3]), s.dropout)
#     reshape(softmax(output), size(output, 1), dims[2], dims[3])
# end

# # Generator forward pass using only Z with argmax and Inputfeeding, size(Z) -> zsize, batchsize, 1
# function (s::GenModel)(Z)
#     s.rnn.h, s.rnn.c = 0, 0
#     input = s.projection(fill(s.charset.eow, size(Z, 2), 1), Z)

#     scores = []
#     for i in 1:s.maxlength
#         rnn_out = s.rnn(input)
#         dims = size(rnn_out)
#         output = s.projection.w' * reshape(rnn_out, dims[1], dims[2] * dims[3])
#         push!(scores, reshape(output, size(output, 1), dims[2], dims[3]))
#         input = vcat(s.projection(softmax(scores[end])), Z)
#     end

#     scores = cat(scores...;dims=3)
# end

# Generator forward pass using Z and Teacher forcing for input
function (s::GenModel)(GenInput) # tuple (input, Z)
    (input, _), Z = GenInput
    s.rnn.h, s.rnn.c = 0, 0
    input = s.embed(input, Z)
    rnn_out = s.rnn(input)
    dims = size(rnn_out)
    output = s.projection.w' * reshape(rnn_out, dims[1], dims[2] * dims[3])
    scores = reshape(output, size(output, 1), dims[2], dims[3])
end

# Generator loss
function (s::GenModel)(GenInput, calculateloss::Int; average=true)
    # since the discriminator will output 2 for the fake data, 
    #    we train the generator to get 1 as output from the discriminator
    (_, output), Z = GenInput
    y = Array(ones(Int, size(Z, 2), s.maxlength)) 
    x = s(GenInput)
    
#     nll(x, mask(output, s.charset.eow); dims=1, average=average)
    
    scores = reshape(x, :, size(output, 1) * size(output, 2))
    output = mask(reshape(output, size(output, 1) * size(output, 2)), s.charset.eow)
    
    dlosses = s.disModel(softmax(x), y, 1) 
    glosses = [ nll(scores[:, i], output[i:i]) for i in 1:size(output, 1) ]
    
    totalloss = glosses .* value(dlosses)
    average && return mean(totalloss)
    return sum(totalloss), length(totalloss)
end

function generate(s::GenModel; start="", maxlength=30)
    s.rnn.h, s.rnn.c = 0, 0
    Z = get_z(s.zsize, 1, 1)
    chars = fill(s.charset.eow, 1)

    starting_index = 1
    for i in 1:length(start)
        push!(chars, s.charset.c2i[start[i]])
        charembed = s.embed(chars[i:i], Z)
        rnn_out = s.rnn(charembed)
        starting_index += 1
    end
    
    for i in starting_index:maxlength
        charembed = s.embed(chars[i:i], Z)
        rnn_out = s.rnn(charembed)
        dims = size(rnn_out)
        output = s.projection.w' * reshape(rnn_out, dims[1], dims[2] * dims[3])
        push!(chars, s.charset.c2i[ sample(s.charset.i2c, Weights(Array(softmax(reshape(output, length(s.charset.i2c)))))) ] )
#         push!(chars, argmax(output)[1])
        if chars[end] == s.charset.eow
            break
        end
    end
    
    join([ s.charset.i2c[i] for i in chars ], "")
end

generate (generic function with 1 method)

## Word Sampler

Word sampler will be used to train discriminator.
this sampler should take B, T (batchsize, timestep) as parameters
returns (X, Y) tuple 
where X is tensor of size (C, B, T)
and Y is array of size B
B consists of real words and generated words
C charset size where each value is weight of this char
in the case of generated words the generator already gives C, B, T
for real words we need to convert words to C, T arrays
where every character can be represented by one hot vector or by Gumble-Max (which is normalized one hot vector)

In [6]:
struct Sampler
    wordsdata::WordsData
    charset::Charset
    genModel::GenModel
    maxBatchsize::Int
end

Base.IteratorSize(::Type{Sampler}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{Sampler}) = Base.HasEltype()
Base.eltype(::Type{Sampler}) = Tuple{KnetArray{Float32,3},Array{Int64,2}}

function Base.iterate(s::Sampler, state=nothing)
    wdatastate = iterate(s.wordsdata, state)
    wdatastate === nothing && (return nothing)
    
    (bucket, state) = wdatastate
    bsize = length(bucket)
    src_eow = s.charset.eow
    src_lengths = map(x -> length(x), bucket)
    max_length = max(src_lengths...)
#     gsize = 1 + rand(bsize:s.maxBatchsize) - bsize # count of words to be generated
    gsize = bsize
    generated = softmax(s.genModel((arraybatch(s.wordsdata, bucket), get_z(s.genModel.zsize, gsize, 1))))

    to_be_cat = [generated, ]
    for (i, v) in enumerate(bucket)
        tindex = [i for i in 1:length(v)]
        pindex = [i for i in length(v)+1:s.wordsdata.maxlength]
        onehot = KnetArray(zeros(Float32, length(s.charset.c2i), 1, s.wordsdata.maxlength))
        onehot[v, :, tindex] .= 1
        onehot[s.charset.eow, :, pindex] .= 1
        onehot = soften(onehot) # soften one hot vectors elements value
        push!(to_be_cat, onehot)
    end
    x = cat(to_be_cat...;dims=2) # concatenate both generated and sampled words

    y = Array(ones(Int, gsize+bsize, s.wordsdata.maxlength)) # create labels 1 -> real, 2-> not-real
    y[1:gsize, :] = y[1:gsize, :] .+ 1
    
    ind = shuffle(1:gsize+bsize) # used to shuffle the batch
    x, y = x[:, ind, :], y[ind, :]
    return (x,y), state
end

In [7]:
function train!(model, parameters, trn, dev, tst; lr=0.001)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn; lr=lr, params=parameters), seconds=30) do y
        devloss = loss(model, dev)
        tstloss = loss(model, tst)
        if devloss < bestloss
            bestloss, bestmodel = devloss, deepcopy(model)
        end
        println(stderr)
        (dev=devloss, tst=tstloss, mem=Float32(CuArrays.usage[]))
    end
    return bestmodel
end

train! (generic function with 1 method)

In [8]:
char_set = "ABCDEFGHIJKLMNOPRSTUVYZabcdefghijklmnoprstuvyzÇÖÜçöüĞğİıŞş"
tr_charset = Charset(char_set)
datadir = "turkish_word_set"
BATCHSIZE = 256
MAXLENGTH = 15
tr_dev = TextReader("$datadir/dev.tr", tr_charset)
tr_trn = TextReader("$datadir/train.tr", tr_charset)
dtrn = WordsData(tr_trn, batchsize=BATCHSIZE, maxlength=MAXLENGTH, bucketwidth = 1)
ddev = WordsData(tr_dev, batchsize=BATCHSIZE, maxlength=MAXLENGTH, bucketwidth = 1)

Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'Ç', 'Ö', 'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş']

WordsData(TextReader("turkish_word_set/dev.tr", Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1)), 256, 15, false, 1, Array{Any,1}[[], [], [], [], [], [], [], [], [], [], [], [], [], [], []])

In [30]:
EMBEDDING_SIZE =128
DHIDDEN_SIZE = 128
GDROPOUT = 0.1
DDROPOUT = 0.3

dismodel = DisModel(tr_charset, EMBEDDING_SIZE, DHIDDEN_SIZE,(
        Dense(DHIDDEN_SIZE, 64, pdrop=DDROPOUT),
        Dense(64, 2, identity)
        ); dropout=DDROPOUT)

GH_SIZE = 256
Z_SIZE = 128

genmodel = GenModel(EMBEDDING_SIZE, Z_SIZE, GH_SIZE, tr_charset, dismodel, MAXLENGTH; dropout=GDROPOUT, layers=2)
trnsampler = Sampler(dtrn, tr_charset, genmodel, BATCHSIZE * 2)
devsampler = Sampler(ddev, tr_charset, genmodel, BATCHSIZE * 2)

Sampler(WordsData(TextReader("turkish_word_set/dev.tr", Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1)), 256, 15, false, 1, Array{Any,1}[[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]), Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1), GenModel(Embed(P(KnetArray{Float32,2}(128,59))), LSTM(input=256,hidden=256,layers=2,dropout=0.1), 0.1, Charset(Dict{Any,Int64}('ç' => 51,'Ğ' => 54,'E' => 6,'Z' => 24,'o' => 39,'B' => 3,'h' => 32,'i' => 33,'r' => 41,'ğ' => 55…), Any["", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'  …  'Ü', 'ç', 'ö', 'ü', 'Ğ', 'ğ', 'İ', 'ı', 'Ş', 'ş'], 1), Embed(P(KnetArray{Float32,2}(256,59)))

In [ ]:
ctrn = collect(dtrn)
cdev = collect(ddev)
collecttrn = [ ((arraybatch(dtrn, i), get_z(Z_SIZE, size(i, 1), 1)), 1) for i in ctrn ]
collectdev = [ ((arraybatch(ddev, i), get_z(Z_SIZE, size(i, 1), 1)), 1) for i in cdev ]

function gmodel()
    global genmodel
    global collecttrn
    global collectdev
    
    trnxbatches = shuffle!(collecttrn)[1:50]
    devbatches = shuffle!(collectdev)[1:10]
    trnmini = trnxbatches[1:5]

    genmodel = train!(genmodel, params(genmodel)[1:3], trnxbatches, devbatches, trnmini)
end

function dmodel(batches, k)
    global trnsampler
    global devsampler
    global dismodel
    
    ctrn = collect(trnsampler)
    ctrn = shuffle!(ctrn)[k * batches + 1:(k + 1) * batches]
    trnmini = ctrn[1:1]
    dev = collect(devsampler)
    dismodel = train!(dismodel, params(dismodel), ctrn, dev, trnmini) 
end

@info "Started training..."
for k in 1:50
    println("Turn no:", k)
    println("Ex.Generated words: \n", join([ generate(genmodel; maxlength=MAXLENGTH) for i in 1:5 ],"\n"))
    dmodel(30, k % 5)
    gmodel()
end

Turn no:1
Ex.Generated words: 


┌ Info: Started training...
└ @ Main In[31]:30


lıŞlüEykt
ygoFdÇaItjBayaU
AVlĞfTmĞaÖPYĞ
ZfhcnAvachöMdlz
pufBoMÖĞyR



┣▋                   ┫ [3.33%, 1/30, 00:00/00:03, 9.55i/s] (dev = 69.16305f0, tst = 0.69160247f0, mem = 7.0570634f9)
┣████████████████████┫ [100.00%, 30/30, 00:00/00:00, 93.86i/s] (dev = 7.7606063f0, tst = 0.077581756f0, mem = 9.593207f9)

┣▍                   ┫ [2.00%, 1/50, 01:26/01:11:42, 86.03s/i] (dev = 229.5202f0, tst = 114.16728f0, mem = 9.661321f9)
┣▊                   ┫ [4.00%, 2/50, 02:55/01:13:03, 89.27s/i] (dev = 226.77637f0, tst = 112.64827f0, mem = 9.72843f9)
┣█▏                  ┫ [6.00%, 3/50, 04:24/01:13:22, 88.84s/i] (dev = 221.67424f0, tst = 110.31864f0, mem = 9.72843f9)
┣█▌                  ┫ [8.00%, 4/50, 05:53/01:13:36, 89.16s/i] (dev = 214.71277f0, tst = 107.2391f0, mem = 9.327405f9)
┣██                  ┫ [10.00%, 5/50, 07:22/01:13:42, 88.85s/i] (dev = 204.47739f0, tst = 102.88393f0, mem = 9.296466f9)
┣██▊                 ┫ [14.00%, 7/50, 10:20/01:13:47, 88.72s/i] (dev = 169.10661f0, tst = 89.999016f0, mem = 1.0434962f9)
┣███▏                ┫ [16.00%, 8/50, 11

Turn no:2
Ex.Generated words: 
AbğaOkeu
rüğIk
Çar
EvşemGuk
dneıde


...


┣▋                   ┫ [3.33%, 1/30, 00:00/00:04, 8.26i/s] (dev = 0.09564709f0, tst = 0.00095639925f0, mem = 8.1115064f9)
┣████████████████████┫ [100.00%, 30/30, 00:00/00:00, 97.81i/s] (dev = 0.07116331f0, tst = 0.0007115749f0, mem = 9.795326f9)

┣▍                   ┫ [2.00%, 1/50, 01:27/01:12:32, 87.04s/i] (dev = 68.681946f0, tst = 38.06584f0, mem = 9.861347f9)
┣▊                   ┫ [4.00%, 2/50, 02:53/01:11:58, 85.67s/i] (dev = 68.554794f0, tst = 37.865047f0, mem = 9.861344f9)
┣█▏                  ┫ [6.00%, 3/50, 04:05/01:07:58, 71.98s/i] (dev = 68.48376f0, tst = 37.66034f0, mem = 9.86134f9)
┣█▌                  ┫ [8.00%, 4/50, 05:32/01:09:15, 87.72s/i] (dev = 68.46881f0, tst = 37.49553f0, mem = 9.861502f9)
┣██                  ┫ [10.00%, 5/50, 07:01/01:10:07, 88.27s/i] (dev = 68.431114f0, tst = 37.385994f0, mem = 9.578155f9)
┣██▍                 ┫ [12.00%, 6/50, 08:29/01:10:44, 88.56s/i] (dev = 68.373314f0, tst = 37.29741f0, mem = 1.1096388f9)
┣██▊                 ┫ [14.00%, 7/50

Turn no:10
Ex.Generated words: 
ŞfaKEYı
zzaşuşı
NNknbleriğ
rtlerış
örak



┣▋                   ┫ [3.33%, 1/30, 00:00/00:04, 8.37i/s] (dev = 0.07048049f0, tst = 0.00070469105f0, mem = 8.0961577f9)
┣████████████████████┫ [100.00%, 30/30, 00:00/00:00, 98.78i/s] (dev = 0.054052416f0, tst = 0.00054043345f0, mem = 9.581758f9)

┣▍                   ┫ [2.00%, 1/50, 01:29/01:13:47, 88.53s/i] (dev = 44.9211f0, tst = 24.4202f0, mem = 9.652126f9)
┣▊                   ┫ [4.00%, 2/50, 02:57/01:13:52, 88.75s/i] (dev = 44.826733f0, tst = 24.340519f0, mem = 9.652124f9)
┣█▏                  ┫ [6.00%, 3/50, 04:27/01:14:05, 89.42s/i] (dev = 44.57745f0, tst = 24.26809f0, mem = 9.652123f9)
┣█▌                  ┫ [8.00%, 4/50, 05:55/01:13:55, 88.14s/i] (dev = 44.275444f0, tst = 24.24345f0, mem = 9.652123f9)
┣██                  ┫ [10.00%, 5/50, 07:24/01:13:55, 88.71s/i] (dev = 44.519764f0, tst = 24.160992f0, mem = 9.355043f9)
┣██▍                 ┫ [12.00%, 6/50, 08:53/01:13:58, 88.97s/i] (dev = 45.011703f0, tst = 24.15331f0, mem = 1.06866157f9)
┣██▊                 ┫ [14.00%, 7/

Turn no:11
Ex.Generated words: 
Ig
ıka
aN
calımını
tmaalırlan



┣▋                   ┫ [3.33%, 1/30, 00:00/00:04, 7.69i/s] (dev = 0.053585365f0, tst = 0.0005357691f0, mem = 8.3373435f9)
┣████████████████████┫ [100.00%, 30/30, 00:00/00:00, 96.43i/s] (dev = 0.042208757f0, tst = 0.00042201893f0, mem = 9.738936f9)

┣▍                   ┫ [2.00%, 1/50, 01:26/01:11:55, 86.30s/i] (dev = 54.81347f0, tst = 32.24716f0, mem = 9.800655f9)
┣▊                   ┫ [4.00%, 2/50, 02:53/01:11:55, 86.29s/i] (dev = 54.404728f0, tst = 31.9036f0, mem = 9.800654f9)
┣█▏                  ┫ [6.00%, 3/50, 04:20/01:12:09, 87.15s/i] (dev = 54.367985f0, tst = 31.82373f0, mem = 9.800655f9)
┣█▌                  ┫ [8.00%, 4/50, 05:46/01:12:09, 86.53s/i] (dev = 54.466633f0, tst = 31.823177f0, mem = 9.800656f9)
┣██                  ┫ [10.00%, 5/50, 07:13/01:12:12, 86.93s/i] (dev = 54.584908f0, tst = 31.808838f0, mem = 9.541289f9)
┣██▍                 ┫ [12.00%, 6/50, 08:40/01:12:14, 86.86s/i] (dev = 54.60176f0, tst = 31.745102f0, mem = 1.07353734f9)
┣██▊                 ┫ [14.00%, 